Обязательная часть


Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.



Дополнительная часть (необязательная)


Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def habr_web_scraper(request_list: list, pages: int):
    BASE_URL = 'https://habr.com'
    for index, req in enumerate(request_list):
        data = []
        for page in range(pages):
            resp = requests.get(
                BASE_URL + f'/ru/search/page{page + 1}', params=dict(
                    q=req,
                    target_type='posts',
                    order='relevance'
                    )
                )

            soup = BeautifulSoup(resp.text, features="lxml")
            for article_tag in soup.find_all('article'):

                dt = article_tag.find('time').get('datetime')

                h2_tag = article_tag.find('h2')
                title = h2_tag.get_text()
                try:
                    link = BASE_URL + h2_tag.find('a').get('href')
                    article_resp = requests.get(link)
                    article_soup = BeautifulSoup(article_resp.text, features="lxml")
                    article_full_text = article_soup.find('div', attrs={'class': 'tm-article-body'}).get_text().strip()
                except:
                    link = ''
                    article_full_text = ''

                rating = article_tag.find('span', attrs={'class': 'tm-votes-meter__value'}).get_text()

                data.append([dt, title, link, rating, article_full_text])

            X = pd.DataFrame(data, columns=['date', 'title', 'link', 'rating', 'article_full_text'])
            X.drop_duplicates(inplace=True)
            X['date'] = pd.to_datetime(X['date']).dt.date

        print(X.head(10))

In [3]:
habr_web_scraper(['python', 'анализ данных'], 2)

         date                                              title  \
0  2022-01-20  Курс «Python для инженеров». Старт 3 потока 31...   
1  2021-12-13  Жаждущим автоматизации: открытый урок «ChatOps...   
2  2020-04-21  Вышел Python 2.7.18, последний релиз ветки Pyt...   
3  2021-07-06  Python Community Meetup 8/07: видео и материал...   
4  2022-01-13  Открытый урок «Пишем Custom Prometheus Exporte...   
5  2020-12-04  Python как компилируемый статически типизирова...   
6  2022-03-08  Вышел мартовский релиз расширения Python для V...   
7  2020-03-03  В начале этого года Python сместил Java и стал...   
8  2021-11-16  EPAM разработала бесплатный курс по программир...   
9  2022-08-19  Осталась неделя до старта 4 потока Python для ...   

                                                link rating  \
0  https://habr.com/ru/company/southbridge/news/t...    +10   
1  https://habr.com/ru/company/southbridge/news/t...     +9   
2                 https://habr.com/ru/news/t/498364/    +19   